# The distribution of Kcat in GO term， including obtain  EC-Kcat, GO-EC, GO-kcat, GO-level, and statistical analysis of GO-Kcat after shuffle

# EC-Kcat

In [ ]:
import re
import pandas as pd
df=pd.DataFrame()
i=0
for eachdata in open('brenda_download.txt'):
    if re.search('^ID',eachdata):
        if re.search(' ',eachdata):
            EC= eachdata.split(' ')[0].split('\t')[1]
        else:
            EC= eachdata.split('\n')[0].split('\t')[1]
    if re.search('^TN',eachdata):
        if re.search('{',eachdata):
            df.loc[i,'EC']=EC
            kcat=eachdata.split(' {')[0].split('# ')[1]
            #print(eachdata)
            if re.search('-',kcat):
                pass
            else:
                if float(kcat)>0:
                    df.loc[i,'kcat']=kcat
                    df.loc[i,'substrate']=eachdata.split(' {')[1].split('} ')[0]
                    df.loc[i,'reference']=eachdata.split('#')[1]   
                    i=i+1
df.to_csv('ec_kact_substrate_brenda_download.csv', sep=',', header=True, index=False) 
df.head(10)
print('uniEC number:'+str(len(np.unique(df['EC']))))
print('Total number:'+str(len(np.unique(df))))

# GO-EC

In [ ]:
import re
outFile=open('GO_ec.csv','w')
outFile.write('GO,ec\n')
for eachdata in open('ec2go.txt'):
    if re.search('!',eachdata):
        pass
    else:
        data=eachdata.split(' > ')
        ec=data[0].split('EC:')[1]
        GO=data[1].split('; ')[1].split('\n')[0]
        spec=ec.split('.')
        if len(spec)>3:
            outstr=GO+','+ec+'\n'
            outFile.write(outstr)
outFile.close()
GO_ec=pd.read_csv('GO_ec.csv')#
print('uniEC number:'+str(len(np.unique(GO_ec['ec']))))
print('uniGO number:'+str(len(np.unique(GO_ec['GO']))))

GO-EC pairs in E.coli

In [ ]:
GO_ec=pd.read_csv('GO_ec.csv')#
ecoli_ec=pd.read_csv('ecoli_ec.txt',sep='\t')#
for index2, row2 in ecoli_ec.iterrows():
    for index, row in GO_ec.iterrows():
        if row["ec"]==row2["EC"]:
            ecoli_ec.loc[index2,'GO']=row["GO"]
            break
            
ecoli_ec.to_csv('ecoli_EC_GO.csv', header=True, index=False)

# GO-Kcat

In [ ]:
import pandas as pd
import numpy as np

ec_kcat=pd.read_csv('ec_kact_substrate_brenda_download.csv')
GO_ec=pd.read_csv('GO_ec.csv',index_col=0)#

GO_kcat=pd.DataFrame()
for eachgo in list(GO_ec.index):
    if len(GO_ec.loc[eachgo,:])>1:#
        eclist=list(GO_ec.loc[eachgo,'ec'])
        for eachec in list(GO_ec.loc[eachgo,'ec']):
            if eachec in np.unique(ec_kcat['EC']):
                for eachkcat in list(ec_kcat[ec_kcat['EC']==eachec]['kcat']):
                    GO_kcat.loc[i,'GO']=eachgo
                    GO_kcat.loc[i,'ec']=eachec
                    GO_kcat.loc[i,'kcat']=eachkcat
                    i=i+1
    else:
        if GO_ec.loc[eachgo,'ec'] in np.unique(ec_kcat['EC']):
            for eachkcat in list(ec_kcat[ec_kcat['EC']==GO_ec.loc[eachgo,'ec']]['kcat']):
                GO_kcat.loc[i,'GO']=eachgo
                GO_kcat.loc[i,'ec']= GO_ec.loc[eachgo,'ec']
                GO_kcat.loc[i,'kcat']=eachkcat#list(ec_kcat[ec_kcat['EC']==GO_ec.loc[eachgo,'ec']]['kcat'])
                i=i+1
            
GO_kcat.to_csv('GO_kcat.csv', sep=',', header=True, index=False) 
print('uniGO number:'+str(len(np.unique(GO_kcat['GO']))))
print('uniEC number:'+str(len(np.unique(GO_kcat['ec']))))
print('Kcat number:'+str(len(GO_kcat['ec'])))
GO_kcat.head(10)

# GO_level_ano

In [22]:
! pip install goatools

[sudo] password for jupyter2: 


In [21]:
from goatools.rpt.rpt_lev_depth import RptLevDepth
from goatools.obo_parser import GODag
obodag = GODag("go-basic.obo")
rptobj = RptLevDepth(obodag)
rptobj.write_summary_cnts_all()

go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms
Dep <-Depth Counts->  <-Level Counts->
Lev   BP    MF    CC    BP    MF    CC
--- ----  ----  ----  ----  ----  ----
00     1     1     1     1     1     1
01    29    15    21    29    15    21
02   420   146   746   264   126   347
03  2213   869  1070  1272   571   492
04  4856  2098  1360  2375  1538   735
05  7295  5051   696  3702  4815   910
06  7276  1929   229  4477  1830   786
07  4694   727    68  4695   969   598
08  1999   201    10  4209   573   256
09   634    79     1  3513   311    51
10   244    13     0  2402   153     4
11    38    19     0  1516   140     1
12     0     0     0   853    42     0
13     0     0     0   307    35     0
14     0     0     0    66    21     0
15     0     0     0    14     7     0
16     0     0     0     4     1     0


In [6]:
#GO_level_ano
import pandas as pd
GO_level_space_out=pd.DataFrame()
i=0
for eachgo in obodag:
    eg_term = obodag[eachgo]  
    GO_level_space_out.loc[i,'GO']=eachgo
    GO_level_space_out.loc[i,'name']=eg_term.name   
    GO_level_space_out.loc[i,'level']=eg_term.level
    GO_level_space_out.loc[i,'namespace']=eg_term.namespace   
    i=i+1
GO_level_space_out.to_csv('GO_level_space_out.txt', sep='\t', header=True, index=False) 


GO-Kcat-level

In [56]:
import pandas as pd
import numpy as np

GO_kcat=pd.read_csv('GO_kcat.csv',index_col=0) 
GO_kcat=GO_kcat.drop(['ec'], axis=1)
GO_kcat_level_space_list=pd.DataFrame()
i=0
for eachgo in obodag:
    eg_term = obodag[eachgo]   
    if eachgo in list(GO_kcat.index):
        #print(eachgo)
        if len(GO_kcat.loc[eachgo,:])>1:
            Kcatlist=GO_kcat.loc[eachgo,'kcat']
            for eachkcat in Kcatlist:
                GO_kcat_level_space_list.loc[i,'GO']=eachgo
                GO_kcat_level_space_list.loc[i,'name']=eg_term.name   
                GO_kcat_level_space_list.loc[i,'level']=eg_term.level
                GO_kcat_level_space_list.loc[i,'namespace']=eg_term.namespace   
                GO_kcat_level_space_list.loc[i,'kcat']=eachkcat 
                i=i+1
        else:
                GO_kcat_level_space_list.loc[i,'GO']=eachgo
                GO_kcat_level_space_list.loc[i,'name']=eg_term.name   
                GO_kcat_level_space_list.loc[i,'level']=eg_term.level
                GO_kcat_level_space_list.loc[i,'namespace']=eg_term.namespace   
                GO_kcat_level_space_list.loc[i,'kcat']=GO_kcat.loc[eachgo,'kcat'] 
                i=i+1           
    else:
        GO_kcat_level_space_list.loc[i,'GO']=eachgo
        GO_kcat_level_space_list.loc[i,'name']=eg_term.name   
        GO_kcat_level_space_list.loc[i,'level']=eg_term.level
        GO_kcat_level_space_list.loc[i,'namespace']=eg_term.namespace   
        GO_kcat_level_space_list.loc[i,'kcat']=99999999999 #no Kcat use9999999 
        i=i+1
GO_kcat_level_space_list=GO_kcat_level_space_list.drop_duplicates(subset=None,keep='first',inplace=False)
GO_kcat_level_space_list.to_csv('GO_kcat_name_level_space_list.csv', sep=',', header=True, index=False) 
print('uniGO number:'+str(len(np.unique(GO_kcat_level_space_list['GO']))))
print(GO_kcat_level_space_list.shape)
GO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]
print('uniGO number:'+str(len(np.unique(GO_kcat['GO']))))
print(GO_kcat.shape)
GO_kcat_level_space_list.head(10)

,GO,name,level,namespace,kcat
0,GO:0000001,mitochondrion inheritance,5.0,biological_process,1.000000e+11
1,GO:0000002,mitochondrial genome maintenance,5.0,biological_process,1.000000e+11
2,GO:0000003,reproduction,1.0,biological_process,1.000000e+11
3,GO:0000006,high-affinity zinc transmembrane transporter a...,7.0,molecular_function,1.000000e+11
4,GO:0000007,low-affinity zinc ion transmembrane transporte...,7.0,molecular_function,1.000000e+11
5,GO:0000009,"alpha-1,6-mannosyltransferase activity",6.0,molecular_function,1.000000e+11
6,GO:0000010,trans-hexaprenyltranstransferase activity,4.0,molecular_function,1.000000e+11
7,GO:0000011,vacuole inheritance,5.0,biological_process,1.000000e+11
8,GO:0000012,single strand break repair,6.0,biological_process,1.000000e+11
9,GO:0000014,single-stranded DNA endodeoxyribonuclease acti...,5.0,molecular_function,1.000000e+11


# GO_level_Kcat reallocate

how many levels

In [99]:
import pandas as pd
import numpy as np
GO_kcat_level_space_list=pd.read_csv('GO_kcat_name_level_space_list.csv')#
levellist=np.unique(GO_kcat_level_space_list['level'])
levellist=sorted(levellist, reverse=True)
print(levellist)


[11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.0]


Reassign Kcat value for each layer from bottom to vertex

In [151]:
import collections
import copy
eachlevel=11
GO_kcat_level_space_list=pd.read_csv('GO_kcat_name_level_space_list.csv',index_col=0)#
print(eachlevel)
GOlist=list(GO_kcat_level_space_list[GO_kcat_level_space_list['level']==eachlevel].index)
GO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]
print(GO_kcat_level_space_list.shape)
GO_kcat_level_space_list.set_index(["GO"], inplace=True)
GO_kcat_level_space_list['updata']='no'
GO_kcat_level_space_list_use=copy.deepcopy(GO_kcat_level_space_list)
GO_kcat_level_space_list_update=pd.DataFrame()
print(GO_kcat_level_space_list_use.shape)
print(GO_kcat_level_space_list_update.shape)

i=0
deal_GO=[]
for eachgo,value in GO_kcat_level_space_list_use.iterrows():
    #print(eachgo)
    if eachgo not in deal_GO:           
        if eachgo in GOlist:
            if eachgo in list(GO_kcat.index):# contain Kcat in this GO term
                GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                GO_kcat_level_space_list_update.loc[i,'updata']=value['updata']
                i=i+1
            else:
                deal_GO.append(eachgo)
                eg_term = obodag[eachgo]
                if len(eg_term.children)>0:
                    for eachchildren in eg_term.children:
                        #print(eachchildren.id)
                        if eachchildren.id in list(GO_kcat.index):#childnode has Kcat
                            print(eachgo)
                            Kcatlist=GO_kcat_level_space_list_use.loc[eachchildren.id,'kcat']
                            if not isinstance(Kcatlist, collections.Iterable):
                                GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                                GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                                GO_kcat_level_space_list_update.loc[i,'level']=eachlevel
                                GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace']  
                                GO_kcat_level_space_list_update.loc[i,'kcat']=Kcatlist 
                                GO_kcat_level_space_list_update.loc[i,'updata']='yse'
                                i=i+1
                            else:
                                j=0
                                for eachkcat in Kcatlist:
                                    GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                                    GO_kcat_level_space_list_update.loc[i,'name']=value['name'] 
                                    GO_kcat_level_space_list_update.loc[i,'level']=eachlevel
                                    GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace']  
                                    GO_kcat_level_space_list_update.loc[i,'kcat']=eachkcat 
                                    GO_kcat_level_space_list_update.loc[i,'updata']='yse'
                                    i=i+1
                                    j=j+1
                        else:# has child node ,but child node not has Kcat
                            #print('child no kcat'+eachgo)
                            GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                            GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                            GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                            GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                            GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                            GO_kcat_level_space_list_update.loc[i,'updata']=value['updata'] 
                            i=i+1
                else:#no child node
                    GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                    GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                    GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                    GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                    GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                    GO_kcat_level_space_list_update.loc[i,'updata']=value['updata']  
                    i=i+1
        else:#GO term not update
            GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
            GO_kcat_level_space_list_update.loc[i,'name']=value['name']
            GO_kcat_level_space_list_update.loc[i,'level']=value['level']
            GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
            GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
            GO_kcat_level_space_list_update.loc[i,'updata']=value['updata'] 
            i=i+1
            
GO_kcat_level_space_list_update=GO_kcat_level_space_list_update.drop_duplicates(subset=None,keep='first',inplace=False)
GO_kcat_level_space_list_update.to_csv('GO_kcat_level_space_list_update_level11.csv', sep=',', header=True, index=False) 
print(GO_kcat_level_space_list_update.shape)

11
(96428, 5)
(96428, 5)
(0, 0)
(89493, 6)


levle 10-0

In [ ]:
import collections
import copy
for eachlevel in range(11,-1,-1):
    #eachlevel=9
    infilename='GO_kcat_level_space_list_update_level%s.csv'%(eachlevel+1)
    outfilename='GO_kcat_level_space_list_update_level%s.csv'%(eachlevel)
    GO_kcat_level_space_list=pd.read_csv(infilename,index_col=0)#
    GO_kcat_level_space_list=GO_kcat_level_space_list.drop_duplicates(subset=None,keep='first',inplace=False)
    print(eachlevel)
    GOlist=list(GO_kcat_level_space_list[GO_kcat_level_space_list['level']==eachlevel].index)
    GO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]
    #print(GO_kcat_level_space_list.shape)
    GO_kcat_level_space_list_use=copy.deepcopy(GO_kcat_level_space_list)
    GO_kcat_level_space_list_update=pd.DataFrame()
    #print(GO_kcat_level_space_list_use.shape)
    #print(GO_kcat_level_space_list_update.shape)
    i=0
    deal_GO=[]
    for eachgo,value in GO_kcat_level_space_list_use.iterrows():
        #print(eachgo)
        if eachgo not in deal_GO:           
            if eachgo in GOlist:
                if eachgo in list(GO_kcat.index):
                    GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                    GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                    GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                    GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                    GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                    GO_kcat_level_space_list_update.loc[i,'updata']=value['updata']
                    i=i+1
                else:
                    deal_GO.append(eachgo)
                    eg_term = obodag[eachgo]
                    if len(eg_term.children)>0:
                        for eachchildren in eg_term.children:
                            #print(eachchildren.id)
                            if eachchildren.id in list(GO_kcat.index):
                                print('hha'+eachgo+eachchildren.id)
                                Kcatlist=GO_kcat_level_space_list_use.loc[eachchildren.id,'kcat']
                                if not isinstance(Kcatlist, collections.Iterable):
                                    GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                                    GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                                    GO_kcat_level_space_list_update.loc[i,'level']=eachlevel
                                    GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                                    GO_kcat_level_space_list_update.loc[i,'kcat']=Kcatlist 
                                    GO_kcat_level_space_list_update.loc[i,'updata']='yes'
                                    i=i+1
                                else:
                                    j=0
                                    for eachkcat in Kcatlist:
                                        GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                                        GO_kcat_level_space_list_update.loc[i,'name']=value['name'] 
                                        GO_kcat_level_space_list_update.loc[i,'level']=eachlevel
                                        GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace']   
                                        GO_kcat_level_space_list_update.loc[i,'kcat']=eachkcat 
                                        GO_kcat_level_space_list_update.loc[i,'updata']='yes'
                                        i=i+1
                                        j=j+1
                            else:
                                #print('child no kcat'+eachgo)
                                GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                                GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                                GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                                GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                                GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                                GO_kcat_level_space_list_update.loc[i,'updata']=value['updata'] 
                                i=i+1
                    else:
                        GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                        GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                        GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                        GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                        GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                        GO_kcat_level_space_list_update.loc[i,'updata']=value['updata']  
                        i=i+1
            else:
                GO_kcat_level_space_list_update.loc[i,'GO']=eachgo
                GO_kcat_level_space_list_update.loc[i,'name']=value['name']
                GO_kcat_level_space_list_update.loc[i,'level']=value['level']
                GO_kcat_level_space_list_update.loc[i,'namespace']=value['namespace'] 
                GO_kcat_level_space_list_update.loc[i,'kcat']=value['kcat'] 
                GO_kcat_level_space_list_update.loc[i,'updata']=value['updata'] 
                i=i+1

    GO_kcat_level_space_list_update=GO_kcat_level_space_list_update.drop_duplicates(subset=None,keep='first',inplace=False)
    GO_kcat_level_space_list_update.to_csv(outfilename, sep=',', header=True, index=False) 
    print(GO_kcat_level_space_list_update.shape)
    #GO_kcat_level_space_list_update.head(10)


In [3]:
import numpy as np
GO_kcat_level_space_list=pd.read_csv('GO_kcat_level_space_list_update_level0.csv')#
newGO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['updata']=='yes']
newGO_kcat.head(10)
GO_kcat_use=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]#
print('GO number before updating:'+str(len(np.unique(newGO_kcat['GO']))))
print(newGO_kcat.shape)

print('GO number after updating:'+str(len(np.unique(GO_kcat_use['GO']))))
print(GO_kcat_use.shape)

GO number before updating:539
(94437, 6)
GO number after updating:2506
(138371, 6)


get the standard deviation

In [5]:
GO_kcat_level_space_list=pd.read_csv('GO_kcat_level_space_list_update_level0.csv',index_col=0)#

GO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]
GO_kcat=GO_kcat[GO_kcat['namespace']=='molecular_function']
print(GO_kcat.shape)
GO_std_list=pd.DataFrame()
GO_level_std_list=pd.DataFrame()
for eachlevel in np.unique(GO_kcat['level']):
    i=0
    GO_std_list_tmp=pd.DataFrame()
    GO_kcat_level=GO_kcat[GO_kcat['level']==eachlevel]
    for eachgo in np.unique(GO_kcat_level.index):
        Kcatlist=GO_kcat_level.loc[eachgo,'kcat']
        if not isinstance(Kcatlist, collections.Iterable):
            pass
        else:
            #print(GO_kcat_level.loc[eachgo,'kcat'])
            GO_std_list.loc[eachgo,'std']=np.std(GO_kcat_level.loc[eachgo,'kcat'], ddof=1) 
            GO_std_list.loc[eachgo,'level']=eachlevel
            GO_std_list_tmp.loc[eachgo,'std']=np.std(GO_kcat_level.loc[eachgo,'kcat'], ddof=1)          
            i=i+1
    GO_level_std_list.loc[eachlevel,'std']=np.mean(GO_std_list_tmp['std'])
    GO_level_std_list.loc[eachlevel,'term number']=i
    
GO_std_list=GO_std_list.drop_duplicates(subset=None,keep='first',inplace=False)
GO_std_list.to_csv('GO_std_list_mf.csv', sep=',', header=True, index=True) 
GO_level_std_list.to_csv('GO_level_std_list_mf.csv', sep=',', header=True, index=True)
GO_level_std_list

(137959, 5)


,std,term number
0.0,959774.439076,1.0
1.0,242600.750894,4.0
2.0,147651.834420,24.0
3.0,43151.058512,129.0
4.0,16857.145369,369.0
5.0,12258.589541,1181.0
6.0,19380.655054,364.0
7.0,11525.276614,120.0
8.0,387.638748,24.0
9.0,6.253150,5.0


shuffling and t-test

In [6]:
#GO term no shuffle std value
GO_std=pd.read_csv('GO_std_list_mf.csv',index_col=0)#
GO_std.head(10)


,std,level
GO:0003674,959774.439076,0.0
GO:0003824,960395.634449,1.0
GO:0005215,362.383632,1.0
GO:0016209,9634.695465,1.0
GO:0060089,10.290029,1.0
GO:0004133,418.605642,2.0
GO:0004362,203.262141,2.0
GO:0004601,594.692021,2.0
GO:0004690,10.674155,2.0
GO:0004784,42405.954771,2.0


In [6]:
def get_go_std(GO_kcat):
    GO_std_list=pd.DataFrame()
    for eachlevel in np.unique(GO_kcat['level']):
        i=0
        GO_std_list_tmp=pd.DataFrame()
        GO_kcat_level=GO_kcat[GO_kcat['level']==eachlevel]
        for eachgo in np.unique(GO_kcat_level.index):
            Kcatlist=GO_kcat_level.loc[eachgo,'newkcat']
            if not isinstance(Kcatlist, collections.Iterable):
                pass
            else:
                #print(GO_kcat_level.loc[eachgo,'kcat'])
                GO_std_list.loc[eachgo,'std']=np.std(GO_kcat_level.loc[eachgo,'newkcat'], ddof=1)
                GO_std_list.loc[eachgo,'level']=eachlevel
                i=i+1
    return(GO_std_list)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import copy
import collections

GO_kcat_level_space_list=pd.read_csv('GO_kcat_level_space_list_update_level0.csv',index_col=0)#
#GO term no shuffle std value
GO_std=pd.read_csv('GO_std_list_mf.csv',index_col=0)#
GO_std.head(10)

GO_kcat=GO_kcat_level_space_list[GO_kcat_level_space_list['kcat']!=99999999999]#

#1000 times
for i in range(1,1001):
    GO_kcat_use = copy.deepcopy(GO_kcat) 
    GO_kcat_shuffle = shuffle(GO_kcat_use['kcat'])
    GO_kcat_use['newkcat']=GO_kcat_shuffle.values
    GO_std_list=get_go_std(GO_kcat_use)
    random_name='random'+str(i)
    for eachgo in GO_std.index:
        if eachgo in GO_std_list.index:
            GO_std.loc[eachgo,random_name]=GO_std_list.loc[eachgo,'std']
GO_std.to_csv('GO_std_cb_random1000_mf.csv', sep=',', header=True, index=True) 
    
GO_std.head(10)

In [32]:
import numpy as np
from scipy import stats

GO_std_nonan=GO_std.dropna(axis=0,how='any') 

GO_std_static=pd.DataFrame()
for i in range(4,len(GO_std_nonan.columns)):
    t2, p2 = stats.ttest_ind(GO_std_nonan['std'].values,GO_std_nonan[GO_std_nonan.columns[i]].values)
    GO_std_static.loc[GO_std_nonan.columns[i],'pvalue']=p2

GO_std_static.to_csv('GO_std_cb_random1000_mf_static.csv', sep=',', header=True, index=True) 
GO_std_static.head(10)

,pvalue
random1,0.000060
random2,0.000546
random3,0.001189
random4,0.000286
random5,0.000015
random6,0.000002
random7,0.000109
random8,0.000055
random9,0.000147
random10,0.000055


In [36]:
print('Total random: '+str(len(GO_std_static['pvalue'])))
print('pvalue<0.001 random numbers: '+str(len(GO_std_static[GO_std_static['pvalue']<0.001])))
print('pvalue<0.05 random numbers: '+str(len(GO_std_static[GO_std_static['pvalue']<0.05])))

Total random: 1000
pvalue<0.001 random numbers: 895
pvalue<0.05 random numbers: 1000


# Gene_exp_cv_kcat in each GO

gene information about exp,cv

In [7]:
import pandas as pd
import numpy as np
gene_information=pd.read_csv('gene_concat_data.csv',index_col=0)#
print('gene number:'+str(len(np.unique(gene_information.index))))
gene_value=gene_information[gene_information['mean_values']>0]
print('gene with expression value:'+str(len(np.unique(gene_value.index))))
gene_information.head(10)

gene number:4502
gene with expression value:4106


,sd_values,mean_values,CV_values,Gene_Name,LeftEnd,Protein,Function,Description
b0001,1.789767,8.948522,0.200007,thrL,190,thr operon leader peptide,Leader; Amino acid biosynthesis: Threonine,Regulatory leader peptide for thrABC operon
b0002,0.973544,10.627619,0.091605,thrA,337,Bifunctional aspartokinase/homoserine dehydrog...,Enzyme; Amino acid biosynthesis: Threonine,Bifunctional aspartokinase/homoserine dehydrog...
b0003,0.933186,10.045601,0.092895,thrB,2801,homoserine kinase,Enzyme; Amino acid biosynthesis: Threonine,Homoserine kinase
b0004,0.855382,10.303469,0.083019,thrC,3734,L-threonine synthase,Enzyme; Amino acid biosynthesis: Threonine,Threonine synthase
b0005,0.829350,8.950972,0.092655,yaaX,5234,DUF2502 family putative periplasmic protein,Null,DUF2502 family putative periplasmic protein; f...
b0006,0.491844,9.363902,0.052526,yaaA,5683,"peroxide resistance protein, lowers intracellu...",Null,"Peroxide resistance protein, lowers intracellu..."
b0007,0.601004,8.453330,0.071097,yaaJ,6529,putative transporter,Putative transport; Transport of small molecul...,"Putative transporter, sodium:alanine (SAF) sym..."
b0008,0.836033,11.597001,0.072090,talB,8238,transaldolase B,Enzyme; Central intermediary metabolism: Non-o...,Transaldolase B
b0009,0.630774,9.783966,0.064470,mog,9306,molybdochelatase incorporating molybdenum into...,"Transport; Biosynthesis of cofactors, carriers...",Molybdochelatase incorporating molybdenum into...
b0010,0.486001,9.030355,0.053819,satP,9928,succinate-acetate transporter,Null,Succinate-acetate transporter


In [8]:
GO_mraker=pd.read_csv('0.175_MF_gene_marker.csv',index_col=0)
print('GO term with marker:'+str(len(np.unique(GO_mraker.index))))
print('unique marker number:'+str(len(np.unique(GO_mraker['marker_gene']))))
print('unigene uumber:'+str(len(np.unique(GO_mraker['gene']))))
GO_mraker.head(10)

GO term with marker:1705
unique marker number:1209
unigene uumber:1235


,gene,marker_gene,cv_values
term_id,,,
GO:0000014,b2761,b2761,0.086116
GO:0000030,b2257,b2257,0.069096
GO:0000034,b3135,b3135,0.057420
GO:0000035,b1094,b1094,0.045795
GO:0000036,b1094,b1094,0.045795
GO:0000049,b2140,b3056,0.043783
GO:0000062,b0042,b1187,0.044563
GO:0000099,b3943,b3943,0.075624
GO:0000100,b0260,b0260,0.105317
